<a href="https://colab.research.google.com/github/JoanWaweru/Sentiment-Analysis-for-Code-Switched-Language/blob/main/SafaricomSentimentProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 4.9 MB/s 
     |████████████████████████████████| 7.6 MB 19.4 MB/s 
     |████████████████████████████████| 182 kB 16.0 MB/s 


In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import EarlyStoppingCallback

In [3]:
# Read csv file into a pandas dataframe
from google.colab import files
uploaded = files.upload()

Saving processed_batch.csv to processed_batch.csv


In [4]:
# Read data
data = pd.read_csv("processed_batch.csv")
data

,Unnamed: 0,Date,User,Tweet,Classification,Confidence
0,0,2022-10-23 23:37:49+00:00,samsabwa,@waxspell @Safaricom_Care @AskPayPal sponyo al...,negative,0.471398
1,1,2022-10-23 23:36:25+00:00,zac124,@Safaricom_Care @SafaricomPLC \r\n\r\nwhat are...,negative,0.697890
2,2,2022-10-23 23:19:35+00:00,tonui_ke,@Safaricom_Care check DM,positive,0.602789
3,3,2022-10-23 22:14:19+00:00,Safaricom_Care,"@martinkanini Hello Martin, please DM https://...",positive,0.595557
4,4,2022-10-23 22:01:01+00:00,martinkanini,@Safaricom_Care am getting some OTP several of...,positive,0.414787
...,...,...,...,...,...,...
886,894,2022-10-22 12:23:09+00:00,Safaricom_Care,"@machelmitoko Hi Samora, please clarify the se...",positive,0.450029
887,895,2022-10-22 12:21:47+00:00,cheisly_a,@Safaricom_Care @Macha_Icon 😹😹😹😹😹,positive,0.707040
888,896,2022-10-22 12:20:11+00:00,lawiarome,Good afternoon @Safaricom_Care as you celebrat...,positive,0.743602
889,897,2022-10-22 12:18:45+00:00,Safaricom_Care,"@iamNelimaBev @iamNelimaBev : Hello, we are on...",negative,0.509154


In [5]:
# Importing LabelEncoder from Sklearn
# library from preprocessing Module.
from sklearn.preprocessing import LabelEncoder
 
# Creating a instance of label Encoder.
le = LabelEncoder()
 
# Using .fit_transform function to fit label
# encoder and return encoded label
label = le.fit_transform(data['Classification'])
 
# printing label
label

array([0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0,
       0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,

In [6]:
# removing the column 'Classification' from data
# as it is of no use now.
data.drop("Classification", axis=1, inplace=True)
 
# Appending the array to our dataFrame
# with column name 'Purchased'
data["Classification"] = label
 
# printing Dataframe
data

,Unnamed: 0,Date,User,Tweet,Confidence,Classification
0,0,2022-10-23 23:37:49+00:00,samsabwa,@waxspell @Safaricom_Care @AskPayPal sponyo al...,0.471398,0
1,1,2022-10-23 23:36:25+00:00,zac124,@Safaricom_Care @SafaricomPLC \r\n\r\nwhat are...,0.697890,0
2,2,2022-10-23 23:19:35+00:00,tonui_ke,@Safaricom_Care check DM,0.602789,1
3,3,2022-10-23 22:14:19+00:00,Safaricom_Care,"@martinkanini Hello Martin, please DM https://...",0.595557,1
4,4,2022-10-23 22:01:01+00:00,martinkanini,@Safaricom_Care am getting some OTP several of...,0.414787,1
...,...,...,...,...,...,...
886,894,2022-10-22 12:23:09+00:00,Safaricom_Care,"@machelmitoko Hi Samora, please clarify the se...",0.450029,1
887,895,2022-10-22 12:21:47+00:00,cheisly_a,@Safaricom_Care @Macha_Icon 😹😹😹😹😹,0.707040,1
888,896,2022-10-22 12:20:11+00:00,lawiarome,Good afternoon @Safaricom_Care as you celebrat...,0.743602,1
889,897,2022-10-22 12:18:45+00:00,Safaricom_Care,"@iamNelimaBev @iamNelimaBev : Hello, we are on...",0.509154,0


In [7]:
# Define pretrained tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=2)

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier

In [8]:
# ----- 1. Preprocess data -----#
# Preprocess data
X = list(data["Tweet"])
y = list(data["Classification"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)


In [9]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [10]:
# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [11]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
    learning_rate=1e-5,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

In [12]:
# Train pre-trained model
trainer.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 712
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 267
  Number of trainable parameters = 66955010


Step,Training Loss,Validation Loss




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=267, training_loss=0.47618794530518493, metrics={'train_runtime': 1200.2947, 'train_samples_per_second': 1.78, 'train_steps_per_second': 0.222, 'total_flos': 56369017734336.0, 'train_loss': 0.47618794530518493, 'epoch': 3.0})

In [14]:
# ----- 3. Predict -----#
# Load test data
test_data = pd.read_csv("processed_batch.csv")
X_test = list(test_data["Classification"])
X_test_tokenized = tokenizer(X_test, padding=True, truncation=True, max_length=512)


In [15]:
# Create torch dataset
test_dataset = Dataset(X_test_tokenized)


In [53]:
# Load trained model
model_path = 'distilbert-base-uncased'
model = DistilBertForSequenceClassification.from_pretrained(model_path, num_labels=2)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification

In [52]:
# Define test trainer
test_trainer = Trainer(model)


No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [49]:
# Make prediction
raw_pred, _, _ = test_trainer.predict(test_dataset)


***** Running Prediction *****
  Num examples = 891
  Batch size = 8


In [50]:
# Preprocess raw predictions
y_pred = np.argmax(raw_pred, axis=1)